In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import os

## Preprocessing

In [2]:
## set parameters
#ROOT = 'PATH/TO/data/processed/'
ROOT = '/home/kddlab/swyoo/data/'
PATH_TO_TRAIN = ROOT + 'train_data.csv'
PATH_TO_TEST = ROOT + 'test_data.csv'
checkpoint_dir = './checkpoint'
if not os.path.exists(checkpoint_dir): os.mkdir(checkpoint_dir)
        
layers = 1
rnn_size = 100
batch_size = 50
drop_keep_prob = 0.7

n_epochs = 10
learning_rate = 0.001
decay = 0.96
decay_steps = 1e4
grad_cap = 0
print_step = 1e3

In [3]:
## load data
data = pd.read_csv(PATH_TO_TRAIN)
valid = pd.read_csv(PATH_TO_TEST)

In [4]:
data.columns = ['SessionId','ItemId','timestamp'] 
valid.columns = ['SessionId','ItemId','timestamp']

In [5]:
data = data.sort_values(['SessionId', 'timestamp'], ascending=[True,True])
valid = valid.sort_values(['SessionId', 'timestamp'], ascending=[True,True])

In [6]:
len(data)

35788

In [7]:
## check sort data
### preprocessing에서 session, timestamp로 sorting을 하였는데,
### sorting이 중요하기 때문에 한번더 확인해본다.
### data가 클경우 sort에서 시간이 오래 걸리기 때문에 sample check을 수행한다.
def check_data_sort(dt, sample_check=False, sample_size=100):
    if sample_check:
        sess_ids = dt['SessionId'].unique()
        sample_sess_ids = np.random.choice(sess_ids, sample_size, replace=False)
        dt = dt[np.in1d(dt.SessionId, sample_sess_ids)]
#     ordered_dt = dt.sort_values(['SessionId', 'timestamp'])
    ordered_dt = dt.sort_values(['SessionId', 'timestamp'], ascending=[True,True])
    return dt.equals(ordered_dt)

print(check_data_sort(data, sample_check=True))
print(check_data_sort(valid))

True
True


In [8]:
## add item index 
### item id를 0번부터 index를 추가합니다.
itemids = data['ItemId'].unique()
n_items = len(itemids)
itemidmap = pd.Series(data=np.arange(n_items), index=itemids).to_dict()
%time data['ItemIdx'] = data['ItemId'].map(lambda x: itemidmap[x])
data[:5]

CPU times: user 15.5 ms, sys: 3.96 ms, total: 19.5 ms
Wall time: 18.9 ms


,SessionId,ItemId,timestamp,ItemIdx
29972,0041a3dd18ec7,7901298,1541343860,0
29973,0041a3dd18ec7,67361,1541344753,1
6654,0052e90367787,817581,1541215148,2
6655,0052e90367787,817581,1541215148,2
6656,0052e90367787,817581,1541215158,2


In [9]:
## offset sessions
### 각 세션의 시작점의 index list를 만든다.
### 즉, 첫번째 sessionid 6의 시작점은 0이고 21의 시작점은 2 이다.
offset_sessions = np.zeros(data['SessionId'].nunique()+1, dtype=np.int32)
offset_sessions[1:] = data.groupby('SessionId').size().cumsum()
offset_sessions[:5]

array([ 0,  2, 35, 38, 50], dtype=int32)

## Prepare Model

In [10]:
## placeholder & learning rate
X = tf.placeholder(tf.int32, [batch_size], name='input')
Y = tf.placeholder(tf.int32, [batch_size], name='output')
States = [tf.placeholder(tf.float32, [batch_size, rnn_size], name='rnn_state') for _ in range(layers)]
global_step = tf.Variable(0, name='global_step', trainable=False)
lr = tf.maximum(1e-5,tf.train.exponential_decay(
    learning_rate, global_step, decay_steps, decay, staircase=True
)) 

## gru weigths
### input item에 대한 embedding matrix 와
### next item 즉 output을 위한 softmax W, b matrix를 구성한다.
with tf.variable_scope('gru_layer', reuse=tf.AUTO_REUSE):
    #sigma = sigma if sigma != 0 else np.sqrt(6.0 / (n_items + rnn_size))
    #initializer = tf.random_uniform_initializer(minval=-sigma, maxval=sigma)
    initializer = tf.glorot_uniform_initializer()
    embedding = tf.get_variable('embedding', [n_items, rnn_size], initializer=initializer)
    softmax_W = tf.get_variable('softmax_w', [n_items, rnn_size], initializer=initializer)
    softmax_b = tf.get_variable('softmax_b', [n_items], initializer=tf.zeros_initializer())
    
## gru_cell
### ㅁt => ㅁt+1 => ㅁt+2 => ... 
### 위와 같은 recurrent network에서 ㅁ. 즉, 단일 gru cell을 말한다.
with tf.variable_scope('gru_cell', reuse=tf.AUTO_REUSE):
    cell = tf.nn.rnn_cell.GRUCell(rnn_size, activation=tf.nn.tanh)
    drop_cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=drop_keep_prob)
    stacked_cell = tf.nn.rnn_cell.MultiRNNCell([drop_cell] * layers)

## feedforward gur_cell
### 예를들어 seesion 1의 item sequence가 5, 7, 9 라면,
### 첫번째 배치에서 아이템 5에 대하여 embedding 하여 inputs를 추출하고,
### session 1의 초기 states로 부터 output과 final_state를 계산한다.
### 두번째 배치에서는 아이템 7에 대한 inputs이 들어가고 아이템 5로 부터 계산된
### final state로 아이템 7에 대한 output과 final_state가 다시 계산된다.
### 즉, 배치 순서로 각 seesion의 item sequence가 recurrent하게 학습되는 것이다.
inputs = tf.nn.embedding_lookup(embedding, X)
output, state_ = stacked_cell(inputs, tuple(States))
final_state = state_

## calculate cost(loss)
### 학습일 경우 negative sampling을 통해 
### cross-entropy loss로 계산하였다. bpt, top1 loss는 주석처리 하였다. 

### for training
sampled_W = tf.nn.embedding_lookup(softmax_W, Y)
sampled_b = tf.nn.embedding_lookup(softmax_b, Y)
logits = tf.matmul(output, sampled_W, transpose_b=True) + sampled_b
### cross-entropy loss
yhat = tf.nn.softmax(logits)
cost = tf.reduce_mean(-tf.log(tf.diag_part(yhat)+1e-24))
### bpr loss
# yhat = logits
# yhatT = tf.transpose(yhat)
# cost = tf.reduce_mean(-tf.log(tf.nn.sigmoid(tf.diag_part(yhat)-yhatT)))
### top1 loss
# yhat = logits
# yhatT = tf.transpose(yhat)
# term1 = tf.reduce_mean(tf.nn.sigmoid(-tf.diag_part(yhat)+yhatT)+tf.nn.sigmoid(yhatT**2), axis=0)
# term2 = tf.nn.sigmoid(tf.diag_part(yhat)**2) / batch_size
# cost = tf.reduce_mean(term1 - term2)

### for prediction
logits_all = tf.matmul(output, softmax_W, transpose_b=True) + softmax_b
yhat_all = tf.nn.softmax(logits_all)

## calculate cost(loss)
### 학습일 경우 negative sampling을 통해 
### cross-entropy loss로 계산하였다. bpt, top1 loss는 주석처리 하였다. 

### for training
sampled_W = tf.nn.embedding_lookup(softmax_W, Y)
sampled_b = tf.nn.embedding_lookup(softmax_b, Y)
logits = tf.matmul(output, sampled_W, transpose_b=True) + sampled_b
### cross-entropy loss
yhat = tf.nn.softmax(logits)
cost = tf.reduce_mean(-tf.log(tf.diag_part(yhat)+1e-24))
### bpr loss
# yhat = logits
# yhatT = tf.transpose(yhat)
# cost = tf.reduce_mean(-tf.log(tf.nn.sigmoid(tf.diag_part(yhat)-yhatT)))
### top1 loss
# yhat = logits
# yhatT = tf.transpose(yhat)
# term1 = tf.reduce_mean(tf.nn.sigmoid(-tf.diag_part(yhat)+yhatT)+tf.nn.sigmoid(yhatT**2), axis=0)
# term2 = tf.nn.sigmoid(tf.diag_part(yhat)**2) / batch_size
# cost = tf.reduce_mean(term1 - term2)

### for prediction
logits_all = tf.matmul(output, softmax_W, transpose_b=True) + softmax_b
yhat_all = tf.nn.softmax(logits_all)

### understanding negative sampling & loss
#### Negative sampling 
위 코드에서 Negative sampling이 어떻게 되는 것인지 이해해보자. <br>
Y 즉, 정답(label) item에 대한 softmax weight 만 추출하여 logit 및 yhat을 계산하고 loss를 계산하는데,
이는 배치 하나의 정답셋(아이템)에 대하여 나머지 item(49개)을 negative로 두어 loss를 계산하는 방식이다.
(나머지 item 중 정답셋과 중복이 되면 어떻게 되나???)
예를 들어 50개의 배치에 rnn out size가 20이라면, output의 shape은 (50, 20)이 될것이다.
sampled_W, sampled_b는 각각 (50, 20), (50,)가 될 것이다. 그리고 output shape은 (50, 50)이 된다.
마지막으로 각 row별 softmax를 취해 yhat을 계산한다.
하나의 row(배치 하나)를 봤을 때, [n, n] 위치가 해당 배치의 정답 item의 yhat 값이 된다.

#### Ranking Loss 
softmax cross-entropy loss는 위와 같이 logits값을 softmax한 후 정답 item의 값에 대해 위와같이 간단하게 계산할 수 있다.
(단점으로 negative item 중 positive item과 중복이 발생할 수 있는데 이는 loss를 크게 만들 것으로 보인다.)
논문에 나오는 BPR loss와 TOP1 loss를 알아보자.

BPR loss <br>
$L_s = - \frac{1}{N_s} \cdot \sum_{j=1}^{N_s} \text{log}(\sigma(\hat{r}_{s,i} -\hat{r}_{s,j}))$ <br>
$N_s$: sample size，$\hat{r}_{s,i}$: yhat of positive sample， $\hat{r}_{s,j}$: yhat of positive sample

postive - negative값에 시그모이드 한 값을 사용하였다. <br>
의미론적으로 보면, pos - neg 값이 클수록 잘 예측된 것이고, 이는 sigmoid 후 1에 가까워 진다. 즉, log변환을 통해 0에 가까워 지므로 적은 loss를 갖는다. 반대로 pos - neg 값이 음수이면 잘 못 예측한 것이고 결국 loss가 커지게 되는 것이다. <br>
코드로 이해하면, tf.diag_part는 logits의 대각행렬 즉, pos 값을 추출한 것이고 shape은 (50,)가 된다. <br>
neg 값과 차를 계산하기 위해 logits 값을 transpose하여 차를 계산하고 sigmoid후 loss를 계산하는 것이다. <br>

TOP loss <br>
$L_s = \frac{1}{N_s} \cdot \sum_{j=1}^{N_s} (\sigma(\hat{r}_{s,j} - \hat{r}_{s,i})) +\sigma(\hat{r^2_j})$ <br>

log를 loss를 빼고, bpr과 반대로 neg - pos 값을 사용하였다. <br>
의미론 적으로, 잘 예측 했다면 pos는 크고 neg는 작기 때문에 음의 값이 될 것이다. sigmoid드 후 0에 가까워 질 것 이다. 즉 첫번째 loss 텀이 0에 가까워 질 것이다. 값을 잘 예측하지 못한다면 양의 값이 될 것이고 loss 텀이 1에 가까워 질 것이다. <br>
만약 pos값을 크게 예측하였다. 하지만, neg값 역시 크게 예측되었다면, 좋은 예측은 아닐 것이다. neg 값은 작아져야 하기 때문에, 이러한 경우에 오른쪽 정규화 텀으로부터 높은 neg값에 penalty를 부여하는 것이다. <br>
위 코드에서 term1이 위와 같은 것이고, term2는 저자 코드를 확인해봐도 사용하고 있는데 계정판에서 추가된 것으로 보인다. <br> term1에서도 정규화 텀에서도 yhatT의 값을 전부 사용했는데, 큰차이가 없어서 그런 것인지? 명확하게는 대각행렬인 pos 값은 제외해야 하는 것이 아닌지 생각이 든다.

In [11]:
## optimize
### Adam optimizer를 사용한다.
optimizer = tf.train.AdamOptimizer(lr)
### grad_cap>0 다면, minimize시 gradient cliping을 수행한다.
### gradient cliping을 수행하는 이유는 다음 블로그 참조 (https://dhhwang89.tistory.com/90)
### 간략하게 학습 중에 gradient가 급격하게 변하는 지점이 발생할 수 있는데, 이는 기존 minima를 찾아가는 방향이 
### 급변할 수 있기 때문에, 이를 방지하기 위해 수행한다.
### 본 학습에서는 cliping을 하지 않는데, 유사하게 learning rate decay을 사용하기 때문인 것으로 생각됨.
tvars = tf.trainable_variables()
gvs = optimizer.compute_gradients(cost, tvars)
if grad_cap > 0:
    capped_gvs = [(tf.clip_by_norm(grad, grad_cap), var) for grad, var in gvs]
else:
    capped_gvs = gvs 
train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)

## Training

In [12]:
## session start
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())

In [13]:
## training
### 위 data feeding에서 in_idx, out_idx 후에 실제 학습을 수행하여, 
### epoch만큼 학습을 진행한다.
tic = time.time()
for epoch in range(n_epochs):
    epoch_cost = []
    state = [np.zeros([batch_size, rnn_size], dtype=np.float32) for _ in range(layers)]
    iters = np.arange(batch_size)
    maxiter = iters.max()
    
    start = offset_sessions[iters]
    end = offset_sessions[iters+1]
    
    finished = False
    while not finished:
        minlen = (end-start).min()
        out_idx = data.ItemIdx.values[start]
        for i in range(minlen-1):
            in_idx = out_idx
            out_idx = data.ItemIdx.values[start+i+1]
            # prepare inputs, targeted outputs and hidden states
            fetches = [cost, final_state, global_step, lr, train_op]
            feed_dict = {X: in_idx, Y: out_idx}
            for j in range(layers): 
                feed_dict[States[j]] = state[j]
            
            cost_, state, step, lr_, _ = sess.run(fetches, feed_dict)
            epoch_cost.append(cost_)
                
            if step == 1 or step % print_step == 0:
                avgc = np.mean(epoch_cost)
                print('Epoch {}\tStep {}\tlr: {:.5f}\tloss: {:.4f}\tElapsed: {:.1f}'.
                      format(epoch, step, lr_, avgc, time.time()-tic))

        start = start+minlen-1
        mask = np.arange(len(iters))[(end-start)<=1]
        for idx in mask:
            maxiter += 1
            if maxiter >= len(offset_sessions)-1:
                finished = True
                break
            iters[idx] = maxiter
            start[idx] = offset_sessions[maxiter]
            end[idx] = offset_sessions[maxiter+1]
        if len(mask):
            for i in range(layers):
                state[i][mask] = 0
        
    avgc = np.mean(epoch_cost)
    if np.isnan(avgc):
        print('Epoch {}: Nan error!'.format(epoch, avgc))
        break
    saver.save(sess, '{}/gru-model'.format(checkpoint_dir), global_step=epoch)
print("1 epoch elapsed time:", time.time() - tic)

Epoch 0	Step 1	lr: 0.00100	loss: 3.9119	Elapsed: 0.1
Epoch 1	Step 1000	lr: 0.00100	loss: 1.9820	Elapsed: 5.5
Epoch 3	Step 2000	lr: 0.00100	loss: 0.4537	Elapsed: 10.9
Epoch 4	Step 3000	lr: 0.00100	loss: 0.1514	Elapsed: 16.2
Epoch 6	Step 4000	lr: 0.00100	loss: 0.0436	Elapsed: 21.6
Epoch 7	Step 5000	lr: 0.00100	loss: 0.0274	Elapsed: 26.9
Epoch 9	Step 6000	lr: 0.00100	loss: 0.0159	Elapsed: 32.3
1 epoch elapsed time: 33.81022095680237


In [14]:
sess.close()

## Prediction & Evaluation
valid(test) 데이터에 대하여 예측과 평가를 수행한다.

evaluation metric
1. Recall@20
    - 예측한 top 20 아이템 중에 정답 아이템이 있는지 1 or 0으로 평가 후 전체를 평균함
2. MRR@20 (mean reciprocal rank)
    - 정답 아이템의 rank의 역수를 취한 후 전체를 평균함
    - $mrr = \frac{1}{N} \sum\limits_{i=1}^{N} {\frac{1}{rank_{i}}}$
    - rank가 20위 안에 들지 않으면 0으로 처리한다.

In [15]:
## parameters
cut_off = 20     # @20
batch_size = 50

In [16]:
## session restore
### 마지막(최신) 학습 checkpoint 정보를 restore한다.
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.latest_checkpoint(checkpoint_dir)
saver.restore(sess, ckpt)

INFO:tensorflow:Restoring parameters from ./checkpoint/gru-model-9


In [17]:
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        
        return False
    
def f1(x):
    if not isfloat(x):
        return  
    elif int(float(x)) not in itemidmap:
        return
    else:
        return int(itemidmap[int(float(x))])

In [18]:
valid['ItemIdx'] = valid['ItemId'].map(f1)
valid = valid.dropna()
valid.ItemIdx = valid.ItemIdx.astype('int') 
display(valid[:2])
print(len(valid))

,SessionId,ItemId,timestamp,ItemIdx
700931,000138ab4f789,152444,1541446323,1782
700937,000138ab4f789,152444,1541447303,1782


413968


In [19]:
## valdation data set
valid['ItemIdx'] = valid['ItemId'].map(lambda x: itemidmap[x])
valid[:5]

,SessionId,ItemId,timestamp,ItemIdx
700931,000138ab4f789,152444,1541446323,1782
700937,000138ab4f789,152444,1541447303,1782
700938,000138ab4f789,152444,1541447344,1782
467565,000195d02a8a9,4028176,1541382865,1488
467566,000195d02a8a9,4028176,1541382874,1488


In [20]:
## valid offset sessions
### 위 학습과 동일하게 각 세션의 시작점의 index list를 만든다.
offset_sessions = np.zeros(valid['SessionId'].nunique()+1, dtype=np.int32)
offset_sessions[1:] = valid.groupby('SessionId').size().cumsum()
offset_sessions[:5]

array([  0,   3,   9, 114, 216], dtype=int32)

In [21]:
## init prediction
### 예측 세션의 배치 사이즈 보다 작을 경우 배치 사이즈를 조정한다.
if len(offset_sessions) - 1 < batch_size:
    batch_size = len(offset_sessions) - 1
### training step과 동일
iters = np.arange(batch_size).astype(np.int32)
maxiter = iters.max()
start = offset_sessions[iters]
end = offset_sessions[iters+1]
in_idx = np.zeros(batch_size, dtype=np.int32)
predict_state = [np.zeros([batch_size, rnn_size], dtype=np.float32) for _ in range(layers)]

In [22]:
## prediction & evaluation
### data feeding 요약
### 학습과는 조금 다르게 valid_mask를 설정하여, batch placeholder에 더이상 feed할 세션이 없어지면,
### 해당 위치를 꺼가는 방식으로 모든 batch placeholder가 없어질 때까지 feed하는 것이다.
evalutation_point_count = 0
mrr, recall = 0.0, 0.0
tic = time.time()
while True:
    ### iters는 batch placeholder로 0보다 큰 즉, 마지막 세션까지는 모든 위치를 켜두고
    ### 아래에서 session 데이터가 다 소진되면 해당 위치를 -1로 할당할 것이다.
    ### valid_mask가 0이 되면 즉 모든 위치가 꺼지면 학습을 종료한다.
    valid_mask = iters >= 0
    if valid_mask.sum() == 0:
        print("break at endpoint", evalutation_point_count)
        break
        
    start_valid = start[valid_mask]
    minlen = (end[valid_mask]-start_valid).min()
    in_idx[valid_mask] = valid.ItemIdx.values[start_valid]
    
    for i in range(minlen-1):
        out_idx = valid.ItemIdx.values[start_valid+i+1]
        ## --- prediction --- ##
        fetches = [yhat_all, final_state]
        feed_dict = {X: in_idx}
        for j in range(layers): 
            feed_dict[States[j]] = predict_state[j]
        preds, predict_state = sess.run(fetches, feed_dict)
        preds = pd.DataFrame(data=np.asarray(preds).T)
        preds.fillna(0, inplace=True) ### preds shape: (item_size, batch_size)
        ## --- evaluation --- ##
        in_idx[valid_mask] = out_idx
        ### 정답 아이템 prediction 값보다 높은 아이템이 몇개인지 카운트 하여 rank를 계산한다.
        ranks = (preds.values.T[valid_mask].T > 
                 np.diag(preds.loc[in_idx].values)[valid_mask]).sum(axis=0) + 1
        ### cutoff에 따른 recall과 mrr을 계산한다.
        rank_ok = ranks < cut_off
        recall += rank_ok.sum()
        mrr += (1.0 / ranks[rank_ok]).sum()
        evalutation_point_count += len(ranks)
        
    start = start+minlen-1
    mask = np.arange(len(iters))[(valid_mask) & (end-start<=1)]
    
    for idx in mask:
        maxiter += 1
        ## 더 이상 할당할 세션이 없으면 해당 위치에 -1을 할당하여 끈다.
        if maxiter >= len(offset_sessions)-1:
            iters[idx] = -1
        else:
            iters[idx] = maxiter
            start[idx] = offset_sessions[maxiter]
            end[idx] = offset_sessions[maxiter+1]
            
    if len(mask):
        for i in range(layers):
            predict_state[i][mask] = 0

### 최종 matric을 계산함.
recall = recall/evalutation_point_count
mrr = mrr/evalutation_point_count
print("recall: ", recall, "mrr:", mrr, "elapsed time:", time.time()-tic)

break at endpoint 391521
recall:  0.6413117048638515 mrr: 0.5364855236310331 elapsed time: 41.28398871421814


In [23]:
### 모든 데이터가 다 사용되었는지 검증.
print(evalutation_point_count)
print(sum(valid.groupby('SessionId').size() - 1))

391521
391521


In [24]:
fetches = [yhat_all, final_state]
feed_dict = {X: in_idx}
for j in range(layers): 
            feed_dict[States[j]] = predict_state[j]        
preds = sess.run(yhat_all, feed_dict)
top_25_idx = np.argsort(preds, axis=1)[...,-25:]
#print(top_25_idx.shape)
clicked_it_idx = in_idx[-1]
submission = np.flip(top_25_idx[-1]) # decending order of impression
print("query: ")
print("clicked_item_id: ", clicked_it_idx)
print("submission: ", submission)

query: 
clicked_item_id:  1134
submission:  [1134  368 2105 1135 1289 1290 2568 1849  196  577 1346  367 1620 1719
   43 1424 2482  901 2493 1094 1642  347 2152 1356 1396]


In [25]:
# sess.close()